In [1]:
import databaseHelper as dbhelper
from fuzzywuzzy import fuzz, process
import datetime

#for connection with Database
dbh = dbhelper.DatabaseHelper()
connection = dbh.createDbConnection("localhost", "admin", "1234","bot")
teams = dbh.getTeamnames(connection)

#Fetch TeamID from Database, needs the Teams already fetched; 
#Optional Teams could be fetched in Method depends on Classstrukture later;
#Home and Awayteam will Come from API or is entered via User
def getTeamIDs(teams,hometeam,awayteam):
    homeTeamID = ""
    awayTeamID = ""

    for i in teams:
        if(fuzz.token_set_ratio(hometeam,i[1]) == 100):
            homeTeamID = i[0]
        elif(fuzz.token_set_ratio(awayteam,i[1]) == 100):
            awayTeamID = i[0]
    
    if(homeTeamID == ""):
        print("Fuck no HomeTeam found")
        return None
    if(awayTeamID == ""):
        print("Fuck no AwayTeam found")
        return None

    return [homeTeamID,awayTeamID]

#returns the Matches that the Teams played against each other.;
#needs the Connection to the Database and the TeamIDArray
def getMatchesOfTeams(connection, teamIDArray):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute("""SELECT * FROM matches WHERE hometeamid=%s and awayteamid=%s or hometeamid=%s and awayteamid=%s"""%(teamIDArray[0],teamIDArray[1],teamIDArray[1],teamIDArray[0]))
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

Database Connection Succesful


In [3]:
teamIDs = getTeamIDs(teams,"Borussia Dortmund", "Bayern Munich")
matches = getMatchesOfTeams(connection, teamIDs)

#Takes TeamID for Team1 and Team2 as Params for Datasorting in Method
#
def dataPreparations(matches, team1, team2):
    if(len(matches) < 8):
        return None
    #All Goals for Goals
    totalGoal = []    
    #Team1 Goals
    team1HomeGoalsFT = []
    team1AwayGoalsFT = []
    team1HomeGoalsHT = []
    team1AwayGoalsHT = []
    team1TotalGoals = []
    #Team1 Cards
    team1YellowCards = []
    team1RedCards = []
    #Team1 Corners
    team1Corners = []
    #Team1 Fouls
    team1Fouls = []
    #Team1 Shots
    team1Shots = []
    team1ShotsOnTarget = []

    #Team2 Goals
    team2HomeGoalsFT = []
    team2AwayGoalsFT = []
    team2HomeGoalsHT = []
    team2AwayGoalsHT = []
    teams2TotalGoals = []
    #Team2 Cards
    team2YellowCards = []
    team2RedCards = []
    #Team2 Corners
    team2Corners = []
    #Team2 Fouls
    team2Fouls = []
    #Team2 Shots
    team2Shots = []
    team2ShotsOnTarget = []
    

    for match in matches:
        #Fulltime Home- and Awaygoal added for later Calculation
        totalGoal.append(match[5])
        totalGoal.append(match[6])
        
        #Team1 is HomeTeam
        if(match[3] == team1 and match[4] == team2):
            team1HomeGoalsFT.append(match[5])
            team1HomeGoalsHT.append(match[8])
            team1YellowCards.append(match[19])
            team1RedCards.append(match[21])
            team1Corners.append(match[17])
            team1Fouls.append(match[15])
            team1Shots.append(match[11])
            team1ShotsOnTarget.append(match[13])
            team2AwayGoalsFT.append(match[6])
            team2AwayGoalsHT.append(match[9])
            team2YellowCards.append(match[20])
            team2RedCards.append(match[22])
            team2Corners.append(match[18])
            team2Fouls.append(match[16])
            team2Shots.append(match[12])
            team2ShotsOnTarget.append(match[14])
        #Team2 is HomeTeam
        elif(match[3] == team2 and match[4] == team1):
            team2HomeGoalsFT.append(match[5])
            team2HomeGoalsHT.append(match[8])
            team2YellowCards.append(match[19])
            team2RedCards.append(match[21])
            team2Corners.append(match[17])
            team2Fouls.append(match[15])
            team2Shots.append(match[11])
            team2ShotsOnTarget.append(match[13])
            team1AwayGoalsFT.append(match[6])
            team1AwayGoalsHT.append(match[9])
            team1YellowCards.append(match[20])
            team1RedCards.append(match[22])
            team1Corners.append(match[18])
            team1Fouls.append(match[16])
            team1Shots.append(match[12])
            team1ShotsOnTarget.append(match[14])

    return {
        'team1': team1,
        'team2': team2,
        'totalGoals': totalGoal,
        'team1HomeGoalsFT': team1HomeGoalsFT,
        'team1HomeGoalsHT': team1HomeGoalsHT,
        'team1AwayGoalsFT': team1AwayGoalsFT,
        'team1AwayGoalsHT': team1AwayGoalsHT,
        'team1YellowCard': team1YellowCards,
        'team1RedCard': team1RedCards,
        'team1Fouls': team1Fouls,
        'team1Corners': team1Corners,
        'team1Shots': team1Shots,
        'team1ShotsOnTarget': team1ShotsOnTarget,
        'team2HomeGoalsFT': team2HomeGoalsFT,
        'team2HomeGoalsHT': team2HomeGoalsHT,
        'team2AwayGoalsFT': team2AwayGoalsFT,
        'team2AwayGoalsHT': team2AwayGoalsHT,
        'team2YellowCard': team2YellowCards,
        'team2RedCard': team2RedCards,
        'team2Fouls': team2Fouls,
        'team2Corners': team2Corners,
        'team2Shots': team2Shots,
        'team2ShotsOnTarget': team2ShotsOnTarget,
    }


In [16]:
def calculateAverage(array):
    ArraySum = sum(c for c in array)
    return ArraySum / len(array)


In [18]:
matches = getMatchesOfTeams(connection, teamIDs)
prepedData = dataPreparations(matches,teamIDs[0],teamIDs[1])
test = "totalGoals"
print(prepedData[test])
team2RedCardTest = calculateAverage(prepedData[test])
print(team2RedCardTest)

[0, 1, 1, 0, 1, 1, 1, 1, 0, 3, 0, 3, 2, 1, 0, 1, 5, 1, 0, 0, 1, 0, 4, 1, 1, 3, 6, 0, 3, 2, 5, 0, 2, 0, 1, 3, 1, 5, 3, 1, 4, 0, 0, 1]
Sum 69
1.5681818181818181


In [7]:
matches = getMatchesOfTeams(connection, teamIDs)
for i in matches:
    if(i[0] == 109):
        matchID = print(i[0])
        division = print(i[1])
        date = print(i[2])
        homeTeamID = print(i[3])
        awayTeamID = print(i[4])
        fullTimeHomeGoal = print(i[5])
        fullTimeAwayGoal = print(i[6])
        fullTimeResult = print(i[7])
        halfTimeHomeGoal = print(i[8])
        halfTimeAwayGoal = print(i[9])
        halfTimeResult = print(i[10])
        HomeTeamShots = print(i[11])
        AwayTeamShots = print(i[12])
        HomeTeamShotTarget = print(i[13])
        AwayTeamShotTarget = print(i[14])
        HomeFoul = print(i[15])
        AwayFoul = print(i[16])
        HomeCorners = print(i[17])
        AwayCorners = print(i[18])
        HomeYellow = print(i[19])
        AwayYellow = print(i[20])
        HomeRed = print(i[21])
        AwayRed = print(i[22])

109
D1
2011-11-19
15
1
0
1
A
0
0
D
16
8
3
2
15
25
3
3
2
1
0
0


In [41]:
import requests
a2010 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2009")
a2011 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2010")
a2012 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2011")
a2013 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2012")
a2014 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2013")
a2015 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2014")
a2016 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2015")
a2017 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2016")
a2018 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2017")
a2019 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2018")
a2020 = requests.get("https://www.openligadb.de/api/getbltable/bl1/2019")


print(a2010.json())
response = [a2011,a2012,a2013,a2014,a2015,a2016,a2017,a2018,a2019,a2020]
points = []
smallestNumber = 1000
biggestNumber = 0

for a in response:
    for i in a.json():
        for k,v in i.items():
            if(k == "Points"):
                points.append(v)

for i in points:
    if (i < smallestNumber):
        smallestNumber = i
    if (i > biggestNumber):
        biggestNumber = i

print(smallestNumber)
print(biggestNumber)

[{'TeamInfoId': 65, 'TeamName': '1. FC Köln', 'ShortName': 'Köln', 'TeamIconUrl': 'https://upload.wikimedia.org/wikipedia/en/thumb/5/53/FC_Cologne_logo.svg/901px-FC_Cologne_logo.svg.png', 'Points': 0, 'OpponentGoals': 0, 'Goals': 0, 'Matches': 0, 'Won': 0, 'Lost': 0, 'Draw': 0, 'GoalDiff': 0}, {'TeamInfoId': 79, 'TeamName': '1. FC Nürnberg', 'ShortName': 'Nürnberg', 'TeamIconUrl': 'https://upload.wikimedia.org/wikipedia/commons/f/fa/1._FC_Nürnberg_logo.svg', 'Points': 0, 'OpponentGoals': 0, 'Goals': 0, 'Matches': 0, 'Won': 0, 'Lost': 0, 'Draw': 0, 'GoalDiff': 0}, {'TeamInfoId': 81, 'TeamName': '1. FSV Mainz 05', 'ShortName': 'Mainz', 'TeamIconUrl': 'https://upload.wikimedia.org/wikipedia/commons/thumb/9/9e/Logo_Mainz_05.svg/1099px-Logo_Mainz_05.svg.png', 'Points': 0, 'OpponentGoals': 0, 'Goals': 0, 'Matches': 0, 'Won': 0, 'Lost': 0, 'Draw': 0, 'GoalDiff': 0}, {'TeamInfoId': 6, 'TeamName': 'Bayer Leverkusen', 'ShortName': 'Leverkusen', 'TeamIconUrl': 'https://upload.wikimedia.org/wikipe

In [36]:
    def updateHomeMatches(connection, data):
        for i in data:
            form = (i[1] / 91) * 100
            insert = """
            UPDATE `matches` SET `hometeamform` = %s WHERE `date` >= "2019-08-16" 
            AND `date` <= "2020-06-27" AND `hometeamid` = %s
            """%(form, i[0])
            dbh.executeQuery(connection, insert)
            
    def updateAwayMatches(connection, data):
        for i in data:
            form = (i[1] / 91) * 100
            second = """
            UPDATE `matches` SET `awayteamform` = %s WHERE `date` >= "2019-08-16" 
            AND `date` <= "2020-06-27" AND `awayteamid` = %s
            """%(form, i[0])
            dbh.executeQuery(connection, second)

In [37]:
tmpResult = []
for i in a2020.json():
    teamName = ""
    points = 0

    for k,v in i.items():
        if(k == "TeamName"):
            teamName = v
        if(k == "Points"):
            points = v
    if(teamName != "" and points != 0):
        tmpResult.append([teamName, points])

print(tmpResult)


[['FC Bayern München', 82], ['BV Borussia Dortmund 09', 69], ['RB Leipzig', 66], ['Borussia Mönchengladbach', 65], ['Bayer Leverkusen', 63], ['TSG 1899 Hoffenheim', 52], ['VfL Wolfsburg', 49], ['SC Freiburg', 48], ['Eintracht Frankfurt', 45], ['Hertha BSC', 41], ['1. FC Union Berlin', 41], ['FC Schalke 04', 39], ['1. FSV Mainz 05', 37], ['1. FC Köln', 36], ['FC Augsburg', 36], ['Werder Bremen', 31], ['Fortuna Düsseldorf', 30], ['SC Paderborn 07', 20]]


In [38]:
from fuzzywuzzy import fuzz
resultHome = []
for i in tmpResult:
    points = 0
    teamId = 0
    if(fuzz.token_set_ratio("Dortmund",i[0]) == 100):
        resultHome.append([1,i[1]])
    if(fuzz.token_set_ratio("Hamburger",i[0]) == 100):
        resultHome.append([2,i[1]])
    if(fuzz.token_set_ratio("Augsburg",i[0]) == 100):
        resultHome.append([3,i[1]])
    if(fuzz.token_set_ratio("Freiburg",i[0]) == 100):
        resultHome.append([4,i[1]])
    if(fuzz.token_set_ratio("Köln",i[0]) == 100):
        resultHome.append([5,i[1]])
    if(fuzz.token_set_ratio("Wolfsburg",i[0]) == 100):
        resultHome.append([6,i[1]])
    if(fuzz.token_set_ratio("Hannover",i[0]) == 100):
        resultHome.append([7,i[1]])
    if(fuzz.token_set_ratio("Hoffenheim",i[0]) == 100):
        resultHome.append([8,i[1]])
    if(fuzz.token_set_ratio("Hertha",i[0]) == 100):
        resultHome.append([9,i[1]])
    if(fuzz.token_set_ratio("Nürnberg",i[0]) == 100):
        resultHome.append([10,i[1]])
    if(fuzz.token_set_ratio("Stuttgart",i[0]) == 100):
        resultHome.append([11,i[1]])
    if(fuzz.token_set_ratio("Schalke",i[0]) == 100):
        resultHome.append([12,i[1]])
    if(fuzz.token_set_ratio("Bremen",i[0]) == 100):
        resultHome.append([13,i[1]])
    if(fuzz.token_set_ratio("Kaiserslautern",i[0]) == 100):
        resultHome.append([14,i[1]])
    if(fuzz.token_set_ratio("München",i[0]) == 100):
        resultHome.append([15,i[1]])
    if(fuzz.token_set_ratio("Mönchengladbach",i[0]) == 100):
        resultHome.append([16,i[1]])
    if(fuzz.token_set_ratio("Mainz",i[0]) == 100):
        resultHome.append([17,i[1]])
    if(fuzz.token_set_ratio("Leverkusen",i[0]) == 100):
        resultHome.append([18,i[1]])
    if(fuzz.token_set_ratio("Düsseldorf",i[0]) == 100):
        resultHome.append([19,i[1]])
    if(fuzz.token_set_ratio("Fürth",i[0]) == 100):
        resultHome.append([20,i[1]])
    if(fuzz.token_set_ratio("Braunschweig",i[0]) == 100):
        resultHome.append([21,i[1]])
    if(fuzz.token_set_ratio("Paderborn",i[0]) == 100):
        resultHome.append([22,i[1]])
    if(fuzz.token_set_ratio("Darmstadt",i[0]) == 100):
        resultHome.append([23,i[1]])
    if(fuzz.token_set_ratio("Ingolstadt",i[0]) == 100):
        resultHome.append([24,i[1]])
    if(fuzz.token_set_ratio("Leipzig",i[0]) == 100):
        resultHome.append([25,i[1]])
    if(fuzz.token_set_ratio("Pauli",i[0]) == 100):
        resultHome.append([26,i[1]])
    if(fuzz.token_set_ratio("Bochum",i[0]) == 100):
        resultHome.append([27,i[1]])
    if(fuzz.token_set_ratio("Union",i[0]) == 100):
        resultHome.append([28,i[1]])
    if(fuzz.token_set_ratio("Frankfurt",i[0]) == 100):
        resultHome.append([29,i[1]])

print(len(resultHome))
updateHomeMatches(connection, resultHome)
updateAwayMatches(connection, resultHome)

18
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
Query succesful
